In [ ]:
%matplotlib inline

In [ ]:
import cf_units
import datetime
import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
#
import umtools.irismode as umu
from umtools.utils import grdstep

In [ ]:
mpl.rcParams['mathtext.default'] = 'regular'

In [ ]:
iris.FUTURE.netcdf_promote = True
um_res = grdstep('km2p2')